In [16]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [17]:
import pickle
import pandas as pd

In [18]:
year = '2022'
month = '02'

In [19]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [20]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    
    return df

In [21]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month}.parquet')

In [22]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [28]:
df['ride_id'] = f'{year}/{month}' + df.index.astype('str')
df['y'] = y_pred

In [32]:
df_result = df[['ride_id', 'y']]

In [34]:
df_result.to_parquet(
    'df.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)